In [1]:
# <문제> 해당 게시물이 영상이어도, 페이지 내에 뜨는 다른 게시물의 첫 번째 이미지가 수집됩니다.

In [2]:
import pandas as pd
import numpy as np
import time
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from urllib.request import urlopen

In [3]:
url = 'https://www.instagram.com/explore/'

In [4]:
#설정값
path = 'C:/Users/ASUX/Desktop/chromedriver_win32/chromedriver.exe' #chromedriver 위치

ID = '' #인스타 id
PW = '' #인스타 비밀번호

SCROLL_MAX = 1000 #얼마나 스크롤할 지 정하기
CRAWLING_NUM = 20 #크롤링할 게시물 개수

In [5]:
def scroll():
    body = driver.find_element_by_tag_name('body')

    while True:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.01) #스크롤 속도 조절
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        
        scroll_len = soup.find(class_='K6yM_').find('div')["style"]
        scroll_len = float(re.search('padding-top: (.*)px;', scroll_len).group(1))
        
        if scroll_len > SCROLL_MAX:
            break

In [6]:
def select_first(driver):
    first = driver.find_element_by_css_selector('._9AhH0')
    first.click()
    time.sleep(1)

In [7]:
img_names = []
img_srcs = []
contents = []
places = []
tags = []

n = 1

data = {'img_name': img_names,
        'img_src': img_srcs,
        'content': contents,
       'place': places,
       'tag': tags,
       }

def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    
    ###이미지
    url = driver.current_url
    driver.execute_script("window.open();") #새 탭에서 열기
    driver.switch_to.window(driver.window_handles[1])
    driver.get(url)
    time.sleep(2)
    
    html_temp = driver.page_source
    soup_temp = BeautifulSoup(html_temp, 'lxml')
    
    # <문제> 해당 게시물이 영상이어도, 페이지 내에 뜨는 다른 게시물의 첫 번째 이미지가 수집됩니다.
    #try:
        #if soup_temp.find("div", class_='fXIG0')['class'] == "fXIG0": #영상 제외
        #    img_srcs.append(np.nan)
        #    img_names.append(np.nan)
        #    driver.close()
        #    driver.switch_to.window(driver.window_handles[0])
            
    #except:
    soup_temp = soup_temp.find("div", class_='KL4Bh')
    img_src = soup_temp.find("img")["src"] #이미지가 두 개이상인 경우도 맨 처음 이미지만 수집됩니다.
    img_srcs.append(img_src)
        
    global n
    img_name = 'img_' + str(n) + '.jpg'
    img_names.append(img_name)
        
    with urlopen(img_src) as f:
        with open('./img/' + 'img_' + str(n) + '.jpg', 'wb') as h:
            img = f.read()
            h.write(img)
        
    n += 1
        
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    
    
    ###내용
    content = soup.select('div.C4VMK > span')[0].text
    if content == '':
        contents.append(np.nan)
    else:
        contents.append(content)
            
            
    ###해시태그
    tag = soup.select('a.xil3i') #댓글에 있는 해시태그도 다 수집되는 것 같아요.
    tags_temp = []
    for i in tag:
        tags_temp.append(i.get_text())
        
    if tags_temp == []:
        tags.append(np.nan)
    else:
        tags.append(''.join(tags_temp))
           
        
    ###위치
    place = soup.select('div.JF9hh')[0].text
    if place == '':
        places.append(np.nan)
    else:
        places.append(place)
            
    
    print("{}개 완료".format(n))
    return data

In [8]:
def move_next(driver):
    right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow') 
    right.click()

In [9]:
driver = webdriver.Chrome(path)
driver.get(url)
time.sleep(1)

In [10]:
#login
login_id = driver.find_element_by_name('username')
login_id.send_keys(ID)
login_pw = driver.find_element_by_name('password')
login_pw.send_keys(PW)
login_pw.send_keys(Keys.RETURN)
time.sleep(3)

In [12]:
#먼저 스크롤 해두기
scroll()
time.sleep(3)

#맨 위로 올라가기
driver.execute_script("window.scrollTo(0, 0);")
time.sleep(3)

#크롤링 시작
select_first(driver)

num = CRAWLING_NUM

for n in range(num):
    data = get_content(driver)
    time.sleep(1)
    move_next(driver)
    time.sleep(1)

1개 완료
2개 완료
3개 완료
4개 완료
5개 완료
6개 완료
7개 완료
8개 완료
9개 완료
10개 완료
11개 완료
12개 완료
13개 완료
14개 완료
15개 완료
16개 완료
17개 완료
18개 완료
19개 완료
20개 완료


In [13]:
df = pd.DataFrame.from_dict(data)

In [14]:
df

,img_name,img_src,content,place,tag
0,img_0.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,요즘 부산 분위기💙#부산여행 #바다 #바다스타그램 #블루라인파크 #💙 #국내여행,NaN,#부산여행#바다#바다스타그램#블루라인파크#💙#국내여행
1,img_1.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,오늘 업뎃된 인기 많은 포비 와이드 밴디...,NaN,NaN
2,img_2.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,가끔 멀리 나가는 것도 조오치 💙🚙,Yongin,NaN
3,img_3.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,Dự là sắp tới Peach sẽ up thật nhiê...,NaN,#uzzlangstyle#koreanstyle#uzzlanggirl#hellowee...
4,img_4.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,[2차 재입고]파이어킹 화이트 스윌 컵앤소서...,NaN,#파이어킹스월#집콕#홈베이킹#빈티지그릇...
5,img_5.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,"#혜정식단 #아침➰#요플레 플레인화이트, 사과➰#요그앤오트 코코넛청크➰#더반한 피칸...",NaN,#혜정식단#아침#요플레#요그앤오트#더반한#어반파머#네이처밸리
6,img_6.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,요즘은 베이스+눈썹+립 끝❕𝖾𝗒𝖾𝖻𝗋𝗈𝗐 @benefi...,NaN,NaN
7,img_7.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,제일 좋아하는 플래브케이크 디자인💖,서울 당산동,NaN
8,img_8.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,Như mọi năm M9 peach nhận đơn order trơ...,NaN,NaN
9,img_9.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,다들 #새해복많이받으세요 🍒.남은 휴일도 즐겁게 보내시고 오늘도 잘부탁드립니다🥰.....,체리오오체리,#새해복많이받으세요#체리오오체리#케이크#주문제작케이크#잠실케이크#잠실새내케이크#송파...
